In [22]:
from datetime import datetime, timedelta
import json
import pandas as pd
from bisect import bisect_left
data = pd.read_excel('./기상청41_단기예보 조회서비스_오픈API활용가이드_241128/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(2411).xlsx')
data.head(3)


,구분,행정구역코드,1단계,2단계,3단계,격자 X,격자 Y,경도(시),경도(분),경도(초),위도(시),위도(분),위도(초),경도(초/100),위도(초/100),위치업데이트
0,kor,1100000000,서울특별시,NaN,NaN,60,127,126,58,48.03,37,33,48.85,126.980008,37.563569,NaN
1,kor,1111000000,서울특별시,종로구,NaN,60,127,126,58,53.91,37,34,13.36,126.981642,37.570378,NaN
2,kor,1111051500,서울특별시,종로구,청운효자동,60,127,126,58,14.35,37,35,2.89,126.970652,37.584137,NaN


In [26]:
serviceKey = "SLnUCvSqwv%2FeT7pxQ2NFc2etMr8%2B7HLq1zXs6Djr1%2BXDUA%2F4YQvKK7C3nhGjIjOX0Vs42IoiIg%2FEQgHpcP371w%3D%3D" # 본인의 서비스 키 입력
# --> 날씨를 알고 싶은 시간 입력
date = (datetime.today() - timedelta(hours=3))
base_date = date.strftime("%Y%m%d") # 발표 일자
base_time = date.strftime("%H00") # 발표 시간
nx = '60' # 예보 지점 x좌표
ny = '127' # 예보 지점 y좌표

In [ ]:
time_list = ["0200", "0500", "0800", "1100", "1400", "1700", "2000", "2300"]
# lower_bound
base_time = time_list[bisect_left(time_list, date.strftime("%H00"))]

In [28]:
base_time

'1700'

In [110]:
# url
#url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0"
#params = {"serviceKey":"SLnUCvSqwv/eT7pxQ2NFc2etMr8+7HLq1zXs6Djr1+XDUA/4YQvKK7C3nhGjIjOX0Vs42IoiIg/EQgHpcP371w==","numOfRows":"1000","pageNo":"1","dataType":"JSON","base_date":input_date,"base_time":input_time,"nx":nx,"ny":ny}
url = f"http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey={serviceKey}&numOfRows=14&pageNo=72&dataType=json&base_date={base_date}&base_time={base_time}&nx={nx}&ny={ny}"
print(url)

http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?serviceKey=SLnUCvSqwv%2FeT7pxQ2NFc2etMr8%2B7HLq1zXs6Djr1%2BXDUA%2F4YQvKK7C3nhGjIjOX0Vs42IoiIg%2FEQgHpcP371w%3D%3D&numOfRows=14&pageNo=72&dataType=json&base_date=20250605&base_time=1700&nx=60&ny=127


In [111]:
# url로 API return값 요청
import requests

requests.get(url)
response = requests.get(url)
res = json.loads(response.text)

In [112]:
res

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20250605',
      'baseTime': '1700',
      'category': 'VEC',
      'fcstDate': '20250609',
      'fcstTime': '1200',
      'fcstValue': '139',
      'nx': 60,
      'ny': 127},
     {'baseDate': '20250605',
      'baseTime': '1700',
      'category': 'WSD',
      'fcstDate': '20250609',
      'fcstTime': '1200',
      'fcstValue': '1',
      'nx': 60,
      'ny': 127},
     {'baseDate': '20250605',
      'baseTime': '1700',
      'category': 'SKY',
      'fcstDate': '20250609',
      'fcstTime': '1200',
      'fcstValue': '1',
      'nx': 60,
      'ny': 127},
     {'baseDate': '20250605',
      'baseTime': '1700',
      'category': 'PTY',
      'fcstDate': '20250609',
      'fcstTime': '1200',
      'fcstValue': '0',
      'nx': 60,
      'ny': 127},
     {'baseDate': '20250605',
      'baseTime': '1700',
      'category': 'POP',
      'fcstD

In [6]:
res['response']['body']['items']['item']

KeyError: 'body'

In [33]:
informations = dict()
for items in res['response']['body']['items']['item'] :
    cate = items['category']
    fcstTime = items['fcstTime']
    fcstValue = items['fcstValue']
    temp = dict()
    temp[cate] = fcstValue
    
    if fcstTime not in informations.keys() :
        informations[fcstTime] = dict()
#     print(items['category'], items['fcstTime'], items['fcstValue'])
#     print(informations[fcstTime])
    informations[fcstTime][cate] = fcstValue

print(informations)

{'0000': {'TMP': '17', 'UUU': '-1.1', 'VVV': '1.5', 'VEC': '141', 'WSD': '1.9', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0100': {'TMP': '16', 'UUU': '-1.8', 'VVV': '2.4', 'VEC': '142', 'WSD': '3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0200': {'TMP': '16', 'UUU': '-1.6', 'VVV': '2.3', 'VEC': '145', 'WSD': '2.9', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '60', 'SNO': '적설없음'}, '0300': {'TMP': '15', 'UUU': '-1.7', 'VVV': '2', 'VEC': '139', 'WSD': '2.7', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '55', 'SNO': '적설없음'}, '0400': {'TMP': '15', 'UUU': '-2', 'VVV': '2.1', 'VEC': '135', 'WSD': '3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': '강수없음', 'REH': '60', 'SNO': '적설없음'}, '0500': {'TMP': '14', 'UUU': '-2.2', 'VVV': '2.4', 'VEC': '136', 'WSD': '3.3', 'SKY': '1', 'PTY': '0', 'POP': '0', 'WAV': '-999', 'PCP': 

In [34]:
deg_code = {0 : 'N', 360 : 'N', 180 : 'S', 270 : 'W', 90 : 'E', 22.5 :'NNE',
           45 : 'NE', 67.5 : 'ENE', 112.5 : 'ESE', 135 : 'SE', 157.5 : 'SSE',
           202.5 : 'SSW', 225 : 'SW', 247.5 : 'WSW', 292.5 : 'WNW', 315 : 'NW',
           337.5 : 'NNW'}

def deg_to_dir(deg) :
    close_dir = ''
    min_abs = 360
    if deg not in deg_code.keys() :
        for key in deg_code.keys() :
            if abs(key - deg) < min_abs :
                min_abs = abs(key - deg)
                close_dir = deg_code[key]
    else : 
        close_dir = deg_code[deg]
    return close_dir
deg_to_dir(0)

'N'

In [35]:
pyt_code = {0 : '강수 없음', 1 : '비', 2 : '비/눈', 3 : '눈', 5 : '빗방울', 6 : '진눈깨비', 7 : '눈날림'}
sky_code = {1 : '맑음', 3 : '구름많음', 4 : '흐림'}


for key, val in zip(informations.keys(), informations.values()) :
#     print(key, val)
    # val['LGT'] -- 낙뢰 
    template = f"""{base_date[:4]}년 {base_date[4:6]}월 {base_date[-2:]}일 {key[:2]}시 {key[2:]}분 {(int(nx), int(ny))} 지역의 날씨는 """ 
    
    
    # 맑음(1), 구름많음(3), 흐림(4)
    if val['SKY'] :
        sky_temp = sky_code[int(val['SKY'])]
#         print("하늘 :", sky_temp)
        template += sky_temp + " "
    
    # (초단기) 없음(0), 비(1), 비/눈(2), 눈(3), 빗방울(5), 빗방울눈날림(6), 눈날림(7)
    if val['PTY'] :
        pty_temp = pyt_code[int(val['PTY'])]
#         print("강수 여부 :",pty_temp)
        template += pty_temp
        # 강수 있는 경우
        if val['RN1'] != '강수없음' :
            # RN1 1시간 강수량 
            rn1_temp = val['RN1']
#             print("강수량(1시간당) :",rn1_temp)
            template += f"시간당 {rn1_temp}mm "
    
    # 기온
    if val['T1H'] :
        t1h_temp = float(val['T1H'])
#         print(f"기온 : {t1h_temp}℃")
        template += f" 기온 {t1h_temp}℃ "
    # 습도
    if val['REH'] :
        reh_temp = float(val['REH'])
#         print(f"습도 : {reh_temp}%")
        template += f"습도 {reh_temp}% "
    # val['UUU'] -- 바람
    
    # val['VVV'] -- 바람
    
    # 풍향/ 풍속
    if val['VEC'] and val['WSD']:
        vec_temp = deg_to_dir(float(val['VEC']))
        wsd_temp = val['WSD']
#         print(f"풍속 :{vec_temp} 방향 {wsd_temp}m/s")
        
    template += f"풍속 {vec_temp} 방향 {wsd_temp}m/s"
    print(template)

KeyError: 'RN1'

In [7]:
import requests
import urllib.parse
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET

# 1) 사용자 설정
service_key = "SLnUCvSqwv%2FeT7pxQ2NFc2etMr8%2B7HLq1zXs6Djr1%2BXDUA%2F4YQvKK7C3nhGjIjOX0Vs42IoiIg%2FEQgHpcP371w%3D%3D"  # ← 여기에 서비스키 입력
nx, ny = 60, 127                     # ← 여기에 조회할 지점 X, Y 격자좌표 입력

# 2) 서비스키 URL 인코딩 (guide 요구사항) :contentReference[oaicite:1]{index=1}
encoded_key = urllib.parse.quote_plus(service_key)

# 3) 최신 발표시각 계산
def get_latest_base(dt):
    slots = [2, 5, 8, 11, 14, 17, 20, 23]
    hour = dt.hour
    candidates = [h for h in slots if h <= hour]
    if not candidates:
        base_date = (dt - timedelta(days=1)).strftime("%Y%m%d")
        base_time = "2300"
    else:
        h = max(candidates)
        base_date = dt.strftime("%Y%m%d")
        base_time = f"{h:02d}00"
    return base_date, base_time

now = datetime.now()  # Asia/Seoul 로컬타임
base_date, base_time = get_latest_base(now)

# 4) API 호출 (serviceKey는 이미 URL에 포함)
base_url = (
    "http://apis.data.go.kr/1360000/"
    "VilageFcstInfoService_2.0/getVilageFcst"
    f"?serviceKey={encoded_key}"
)
params = {
    "pageNo": "1",
    "numOfRows": "1000",
    "dataType": "JSON",      # JSON 요청
    "base_date": base_date,
    "base_time": base_time,
    "nx": nx,
    "ny": ny
}

resp = requests.get(base_url, params=params)
resp.raise_for_status()

# 5) 응답 파싱: JSON 우선, 실패 시 XML
try:
    result = resp.json()
    items = result["response"]["body"]["items"]["item"]
except ValueError:
    # JSON 디코드 실패하면 XML로 파싱
    root = ET.fromstring(resp.text)
    # <item> 요소들을 모두 모아서 dict 형태로 처리
    items = []
    for item_el in root.findall(".//item"):
        it = {child.tag: child.text for child in item_el}
        items.append(it)

# 6) ‘강수형태(PTY)’ 항목에서 비(0 이외) 수집
rain_times = []
for it in items:
    if it.get("category") == "PTY" and it.get("fcstValue") not in (None, "0"):
        dt_str = it["fcstDate"] + it["fcstTime"]
        rain_times.append(datetime.strptime(dt_str, "%Y%m%d%H%M"))

# 7) 결과 출력
if not rain_times:
    print("향후 일주일 내에 비 예보는 없습니다.")
else:
    print("향후 일주일 내 비 예보 시점:")
    for t in sorted(set(rain_times)):
        print("  -", t.strftime("%Y-%m-%d %H:%M"))


향후 일주일 내에 비 예보는 없습니다.


In [6]:
resp

<Response [200]>